In [ ]:
import numpy as np
import tensorflow as tf

load training data from .dat files
raw_data_imaginary_x.dat --> train_imag
raw_data_real_x.dat      --> train_real

In [ ]:
trainingset_number = 0
neuron_number = 8
activation_function = 'tanh'
iterations = 20000


In [ ]:
filename_raw_data_real = "raw_data_real_" + str(trainingset_number) + ".dat"
filename_raw_data_imag = "raw_data_imaginary_" + str(trainingset_number) + ".dat"

# my_file = open(filename_raw_data_real, "r")
# lines_real = my_file.readlines()
# my_file.close()

# my_file = open(filename_raw_data_imag, "r")
# lines_imag = my_file.readlines()
# my_file.close()

# train_real = []
# train_imag = []

# for line in lines_real:
#     train_real = [train_real ,float(line)]

# for line in lines_imag:
#     train_imag = [train_imag ,float(line)]

dataset_real = tf.data.TextLineDataset(filename_raw_data_real)
dataset_imag = tf.data.TextLineDataset(filename_raw_data_imag)

# convert strings to float
def parse_fnc(line):
    string_vals = tf.strings.split([line]).values
    return tf.strings.to_number(string_vals, tf.float32)

dataset_real = dataset_real.map(map_func=parse_fnc)
dataset_imag = dataset_imag.map(map_func=parse_fnc)

# merge datasets
dataset = dataset_real.zip(dataset_imag)


In [53]:
print(dataset)
print(dataset_real)

<ZipDataset element_spec=TensorSpec(shape=(None,), dtype=tf.float32, name=None)>
<MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.float32, name=None)>


In [ ]:
FC_layer = tf.keras.layers.Dense(units=neuron_number, activation=activation_function) # dense layer == fully connected layer

input_layer = tf.keras.layers.Dense(units=2, input_dim=2, activation=activation_function)
output_layer = tf.keras.layers.Dense(units=2, activation=activation_function)

In [ ]:
model = tf.keras.Sequential([input_layer, FC_layer, output_layer])


In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = iterations
# fit the keras model on the dataset
model.fit(dataset)

In [ ]:
# training progress
# diagram number 2 paper page 4

# cost function mean square error